In [6]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import gc

import keras as k
import tensorflow as tf
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, Activation, BatchNormalization
from keras.callbacks import EarlyStopping
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input
from keras import backend as K
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import cv2
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score, precision_score 
from skimage import io,transform

import time

import os
import fnmatch

In [7]:
datagen = ImageDataGenerator(
    rotation_range=90,
    fill_mode='reflect',
    horizontal_flip=True,
    vertical_flip=True)

def multilabelmetrics(y_true,y_pred):
    '''y_true and y_pred should be boolean np arrays
    of shape num_example x num_classes '''
    total = np.sum(y_true,axis = 0)
    tp = np.sum(y_true*y_pred,axis=0)
    tn = np.sum((1-y_true)*(1-y_pred),axis=0)
    fp = np.sum((1-y_true)*y_pred,axis=0)
    fn = np.sum(y_true*(1-y_pred),axis=0)
    return total,tp,tn,fp,fn

def combine_predictions(x,y,y1,y2,thresh,thresh1,thresh2,thresh3):
    y_pred = np.zeros((x.shape[0],17),np.uint8)
    y_bool = np.array((y > thresh),np.uint8)
    y1_bool = np.array((y1 > thresh1),np.uint8)
    y2_bool = np.array((y2 > thresh2)*np.tile(y1[:,0]>thresh3,(7,1)).T,np.uint8)
    y_pred[:,:7] = y2_bool
    y_pred[:,7:13] = y1_bool[:,1:]
    y_pred[:,13:] = y_bool
    return y_pred

def combine_predictions_2(x,y,y1,y2,thresh,thresh1,thresh2,thresh3,thresh4):#if cloud > thresh4, 
    y_pred = np.zeros((x.shape[0],17),np.uint8)
    y_bool = np.array((y > thresh),np.uint8)
    y1_bool = np.array((y1 > thresh1),np.uint8)
    y2_bool = np.array((y2 > thresh2)*np.tile(y1[:,0]>thresh3,(7,1)).T,np.uint8)
    y_pred[:,:7] = y2_bool
    y_pred[:,7:13] = y1_bool[:,1:]
    y_pred[:,13:] = y_bool
    y_pred[((np.nonzero(y[:,1]>thresh4))[0]),:] = 0
    y_pred[((np.nonzero(y[:,1]>thresh4))[0]),14] = 1
    return y_pred

callbacks = [EarlyStopping(monitor='val_loss', patience=2, verbose=0)]

In [8]:
x_train = np.zeros((40479,32,32,3), np.float32)
y_train = []

df_train = pd.read_csv('train_v2.csv')

labels = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'artisinal_mine',
 'selective_logging',         
 'slash_burn', 
 'cultivation',
 'habitation',
 'road',
 'water',
 'haze',
 'partly_cloudy', 
 'cloudy',
 'agriculture',
 'clear',
 'primary',]

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

i=0

for f, tags in tqdm(df_train.values[:40479], miniters=1000):    
    img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train[i,:,:,:] = np.array(cv2.resize(img, (32, 32)),np.float32)/255.#139 minimum size for inception
    i+=1
    y_train.append(targets)
  
y_train = np.array(y_train, np.uint8)

print(x_train.shape)
print(y_train.shape)

100%|██████████| 40479/40479 [00:53<00:00, 760.10it/s]

(40479, 32, 32, 3)
(40479, 17)


In [9]:
#subtracting mean
train_mean = np.mean(x_train,axis = 0)
x_train -= train_mean

In [10]:
#last three labels +others
y_train_1 = np.zeros((y_train.shape[0],4))
y_train_1[:,1:] = y_train[:,-3:]
y_train_1[:,0] = (np.sum(y_train[:,:-3],axis=1)>0)
y_train_1 = np.array(y_train_1,np.uint8)
x_train, x_val, y_train_1, y_val_1 = train_test_split(x_train,y_train_1,test_size=0.1)
print(x_train.shape)
print(y_train_1.shape)
print(x_val.shape)
print(y_val_1.shape)

(36431, 32, 32, 3)
(36431, 4)
(4048, 32, 32, 3)
(4048, 4)


In [18]:
model = Sequential()#using same architecture for all three models
model.add(Conv2D(32, (3, 3), padding = 'same', input_shape=(32, 32, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(48, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(48, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(48, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(128, (3, 3), padding = 'same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(2048))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(1024))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dense(4, activation='sigmoid'))

In [19]:
model.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])    
model.fit_generator(datagen.flow(x_train,y_train_1, batch_size = 128), validation_data=(x_val, y_val_1),
                  epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
285/284 [==============================] - 52s - loss: 0.2859 - acc: 0.8812 - val_loss: 0.7289 - val_acc: 0.7474
Epoch 2/10
285/284 [==============================] - 50s - loss: 0.2363 - acc: 0.9038 - val_loss: 0.2709 - val_acc: 0.8896
Epoch 3/10
285/284 [==============================] - 50s - loss: 0.2200 - acc: 0.9118 - val_loss: 0.2151 - val_acc: 0.9130
Epoch 4/10
285/284 [==============================] - 50s - loss: 0.2133 - acc: 0.9139 - val_loss: 0.2211 - val_acc: 0.9098
Epoch 5/10
285/284 [==============================] - 50s - loss: 0.2053 - acc: 0.9174 - val_loss: 0.2018 - val_acc: 0.9216
Epoch 6/10
285/284 [==============================] - 50s - loss: 0.2030 - acc: 0.9184 - val_loss: 0.2196 - val_acc: 0.9156
Epoch 7/10
285/284 [==============================] - 50s - loss: 0.1997 - acc: 0.9192 - val_loss: 0.2430 - val_acc: 0.8991
Epoch 8/10
285/284 [==============================] - 50s - loss: 0.1961 - acc: 0.9219 - val_loss: 0.1971 - val_acc: 0.9172
Epoch 9/

In [20]:
y_pred = model.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_1, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.939388471976
thresh: 0.1 	F2 score: 0.950734733453
thresh: 0.15 	F2 score: 0.955401082536
thresh: 0.2 	F2 score: 0.957815327463
thresh: 0.25 	F2 score: 0.957823675256
thresh: 0.3 	F2 score: 0.958387797976
thresh: 0.35 	F2 score: 0.957659220208


In [21]:
#continue with reduced learning rate
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0005),
              metrics=['accuracy']) 
model.fit_generator(datagen.flow(x_train,y_train_1, batch_size = 128), validation_data=(x_val, y_val_1),
                  epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
285/284 [==============================] - 52s - loss: 0.1877 - acc: 0.9243 - val_loss: 0.1852 - val_acc: 0.9262
Epoch 2/10
285/284 [==============================] - 51s - loss: 0.1861 - acc: 0.9253 - val_loss: 0.1798 - val_acc: 0.9262
Epoch 3/10
285/284 [==============================] - 50s - loss: 0.1838 - acc: 0.9263 - val_loss: 0.1744 - val_acc: 0.9296
Epoch 4/10
285/284 [==============================] - 51s - loss: 0.1831 - acc: 0.9262 - val_loss: 0.1745 - val_acc: 0.9280
Epoch 5/10
285/284 [==============================] - 50s - loss: 0.1815 - acc: 0.9269 - val_loss: 0.1724 - val_acc: 0.9294
Epoch 6/10
285/284 [==============================] - 50s - loss: 0.1806 - acc: 0.9273 - val_loss: 0.1761 - val_acc: 0.9295
Epoch 7/10
285/284 [==============================] - 50s - loss: 0.1789 - acc: 0.9275 - val_loss: 0.1840 - val_acc: 0.9258
Epoch 8/10
285/284 [==============================] - 50s - loss: 0.1782 - acc: 0.9283 - val_loss: 0.1738 - val_acc: 0.9290


In [22]:
y_pred = model.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_1, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.951457165947
thresh: 0.1 	F2 score: 0.959909379277
thresh: 0.15 	F2 score: 0.961930620102
thresh: 0.2 	F2 score: 0.962087778826
thresh: 0.25 	F2 score: 0.960786525338
thresh: 0.3 	F2 score: 0.959661004316
thresh: 0.35 	F2 score: 0.957541742522


In [23]:
#continue with reduced learning rate
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0002),
              metrics=['accuracy']) 
model.fit_generator(datagen.flow(x_train,y_train_1, batch_size = 128), validation_data=(x_val, y_val_1),
                  epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
285/284 [==============================] - 53s - loss: 0.1720 - acc: 0.9309 - val_loss: 0.1666 - val_acc: 0.9313
Epoch 2/10
285/284 [==============================] - 50s - loss: 0.1711 - acc: 0.9310 - val_loss: 0.1675 - val_acc: 0.9315
Epoch 3/10
285/284 [==============================] - 50s - loss: 0.1706 - acc: 0.9305 - val_loss: 0.1654 - val_acc: 0.9329
Epoch 4/10
285/284 [==============================] - 50s - loss: 0.1699 - acc: 0.9306 - val_loss: 0.1642 - val_acc: 0.9316
Epoch 5/10
285/284 [==============================] - 51s - loss: 0.1694 - acc: 0.9315 - val_loss: 0.1668 - val_acc: 0.9320
Epoch 6/10
285/284 [==============================] - 50s - loss: 0.1690 - acc: 0.9320 - val_loss: 0.1655 - val_acc: 0.9316
Epoch 7/10
285/284 [==============================] - 50s - loss: 0.1679 - acc: 0.9318 - val_loss: 0.1640 - val_acc: 0.9322
Epoch 8/10
285/284 [==============================] - 50s - loss: 0.1675 - acc: 0.9320 - val_loss: 0.1656 - val_acc: 0.9329
Epoch 9/

In [24]:
y_pred = model.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_1, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.951512547839
thresh: 0.1 	F2 score: 0.959727880378
thresh: 0.15 	F2 score: 0.963224139882
thresh: 0.2 	F2 score: 0.964286186937
thresh: 0.25 	F2 score: 0.963844261647
thresh: 0.3 	F2 score: 0.963093951943
thresh: 0.35 	F2 score: 0.960863067745


In [25]:
#continue with reduced learning rate
model.compile(optimizer=Adam(lr=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train,y_train_1, batch_size = 128), validation_data=(x_val, y_val_1),
                  epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
285/284 [==============================] - 53s - loss: 0.1657 - acc: 0.9324 - val_loss: 0.1623 - val_acc: 0.9334
Epoch 2/10
285/284 [==============================] - 50s - loss: 0.1635 - acc: 0.9334 - val_loss: 0.1637 - val_acc: 0.9331
Epoch 3/10
285/284 [==============================] - 50s - loss: 0.1624 - acc: 0.9342 - val_loss: 0.1604 - val_acc: 0.9350
Epoch 4/10
285/284 [==============================] - 50s - loss: 0.1622 - acc: 0.9345 - val_loss: 0.1600 - val_acc: 0.9346
Epoch 5/10
285/284 [==============================] - 50s - loss: 0.1635 - acc: 0.9334 - val_loss: 0.1610 - val_acc: 0.9352
Epoch 6/10
285/284 [==============================] - 50s - loss: 0.1626 - acc: 0.9334 - val_loss: 0.1617 - val_acc: 0.9347
Epoch 7/10
285/284 [==============================] - 50s - loss: 0.1620 - acc: 0.9343 - val_loss: 0.1606 - val_acc: 0.9345


In [26]:
y_pred = model.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_1, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.95678010994
thresh: 0.1 	F2 score: 0.963187576356
thresh: 0.15 	F2 score: 0.963757794741
thresh: 0.2 	F2 score: 0.964217239959
thresh: 0.25 	F2 score: 0.963596096492
thresh: 0.3 	F2 score: 0.963189944752
thresh: 0.35 	F2 score: 0.96209597896


In [27]:
#continue with reduced learning rate
model.compile(optimizer=Adam(lr=0.00005), loss='binary_crossentropy', metrics=['accuracy'])
model.fit_generator(datagen.flow(x_train,y_train_1, batch_size = 128), validation_data=(x_val, y_val_1),
                  epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
285/284 [==============================] - 53s - loss: 0.1604 - acc: 0.9346 - val_loss: 0.1602 - val_acc: 0.9346
Epoch 2/10
285/284 [==============================] - 50s - loss: 0.1605 - acc: 0.9345 - val_loss: 0.1612 - val_acc: 0.9335
Epoch 3/10
285/284 [==============================] - 51s - loss: 0.1606 - acc: 0.9342 - val_loss: 0.1593 - val_acc: 0.9352
Epoch 4/10
285/284 [==============================] - 51s - loss: 0.1602 - acc: 0.9349 - val_loss: 0.1599 - val_acc: 0.9337
Epoch 5/10
285/284 [==============================] - 51s - loss: 0.1597 - acc: 0.9349 - val_loss: 0.1592 - val_acc: 0.9348
Epoch 6/10
285/284 [==============================] - 50s - loss: 0.1603 - acc: 0.9346 - val_loss: 0.1612 - val_acc: 0.9341
Epoch 7/10
285/284 [==============================] - 50s - loss: 0.1591 - acc: 0.9349 - val_loss: 0.1606 - val_acc: 0.9349
Epoch 8/10
285/284 [==============================] - 50s - loss: 0.1586 - acc: 0.9358 - val_loss: 0.1598 - val_acc: 0.9349


In [28]:
y_pred = model.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_1, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.955327973884
thresh: 0.1 	F2 score: 0.962399867147
thresh: 0.15 	F2 score: 0.964426179859
thresh: 0.2 	F2 score: 0.965192494934
thresh: 0.25 	F2 score: 0.965051124649
thresh: 0.3 	F2 score: 0.964377899819
thresh: 0.35 	F2 score: 0.962920904117


In [29]:
model.save("32_net1")
model.save_weights("32_net1_weights")

In [30]:
num_2 = np.sum(np.sum(y_train[:,:-3],axis=1)>0)
print(num_2)

25171


In [31]:
x_val = []
x_train = []
x_train = np.zeros((num_2,32,32,3), np.float32)
y_train = []

df_train = pd.read_csv('train_v2.csv')

labels = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'artisinal_mine',
 'selective_logging',         
 'slash_burn', 
 'cultivation',
 'habitation',
 'road',
 'water',
 'haze',
 'partly_cloudy', 
 'cloudy',
 'agriculture',
 'clear',
 'primary',]

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

i=0

for f, tags in tqdm(df_train.values[:40479], miniters=1000):    
    img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    if(np.sum(targets[:-3])>0):
        x_train[i,:,:,:] = np.array(cv2.resize(img, (32, 32)),np.float32)/255.#139 minimum size for inception
        i+=1
        y_train.append(targets)
    
y_train = np.array(y_train, np.uint8)

print(x_train.shape)
print(y_train.shape)

100%|██████████| 40479/40479 [00:54<00:00, 738.66it/s]

(25171, 32, 32, 3)
(25171, 17)


In [32]:
y_train_2 = np.zeros((y_train.shape[0],8))
y_train_2[:,1:] = y_train[:,-10:-3]
y_train_2[:,0] = (np.sum(y_train[:,:-10],axis=1)>0)
y_train_2 = np.array(y_train_2,np.uint8)

In [33]:
print(y_train[100,:])
print(y_train_2[100,:])

[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1]
[0 0 0 0 0 1 0 0]


In [34]:
x_train -= train_mean
x_train, x_val, y_train_2, y_val_2 = train_test_split(x_train,y_train_2,test_size=0.1)
print(x_train.shape)
print(y_train_2.shape)
print(x_val.shape)
print(y_val_2.shape)

(22653, 32, 32, 3)
(22653, 8)
(2518, 32, 32, 3)
(2518, 8)


In [35]:
#model1
from keras.models import load_model
model1 = load_model("32_net1")
model1.pop()
model1.add(Dense(8, activation='sigmoid'))

for layer in model1.layers[:-1]:
    layer.trainable = False

In [36]:
model1.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

model1.fit_generator(datagen.flow(x_train,y_train_2, batch_size = 128), validation_data=(x_val, y_val_2),
                   epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
177/176 [==============================] - 14s - loss: 0.3110 - acc: 0.8675 - val_loss: 0.2641 - val_acc: 0.8878
Epoch 2/10
177/176 [==============================] - 11s - loss: 0.2678 - acc: 0.8859 - val_loss: 0.2572 - val_acc: 0.8904
Epoch 3/10
177/176 [==============================] - 11s - loss: 0.2625 - acc: 0.8875 - val_loss: 0.2549 - val_acc: 0.8919
Epoch 4/10
177/176 [==============================] - 11s - loss: 0.2600 - acc: 0.8891 - val_loss: 0.2525 - val_acc: 0.8939
Epoch 5/10
177/176 [==============================] - 11s - loss: 0.2580 - acc: 0.8897 - val_loss: 0.2509 - val_acc: 0.8933
Epoch 6/10
177/176 [==============================] - 11s - loss: 0.2580 - acc: 0.8895 - val_loss: 0.2513 - val_acc: 0.8908
Epoch 7/10
177/176 [==============================] - 11s - loss: 0.2557 - acc: 0.8906 - val_loss: 0.2485 - val_acc: 0.8944
Epoch 8/10
177/176 [==============================] - 11s - loss: 0.2554 - acc: 0.8906 - val_loss: 0.2495 - val_acc: 0.8927
Epoch 9/

In [37]:
y_pred = model1.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_2, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.739930490467
thresh: 0.1 	F2 score: 0.770330021322
thresh: 0.15 	F2 score: 0.779769878305
thresh: 0.2 	F2 score: 0.775405938237
thresh: 0.25 	F2 score: 0.761122709817
thresh: 0.3 	F2 score: 0.74102797253
thresh: 0.35 	F2 score: 0.717149168044


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [38]:
for layer in model1.layers[:-1]:
    layer.trainable = True
    
model1.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics=['accuracy']) 
model1.fit_generator(datagen.flow(x_train,y_train_2, batch_size = 128), validation_data=(x_val, y_val_2),
                   epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
177/176 [==============================] - 35s - loss: 0.2504 - acc: 0.8946 - val_loss: 0.3112 - val_acc: 0.8720
Epoch 2/10
177/176 [==============================] - 31s - loss: 0.2405 - acc: 0.9001 - val_loss: 0.2506 - val_acc: 0.8967
Epoch 3/10
177/176 [==============================] - 31s - loss: 0.2346 - acc: 0.9027 - val_loss: 0.2676 - val_acc: 0.8909
Epoch 4/10
177/176 [==============================] - 31s - loss: 0.2319 - acc: 0.9046 - val_loss: 0.2432 - val_acc: 0.9038
Epoch 5/10
177/176 [==============================] - 31s - loss: 0.2288 - acc: 0.9052 - val_loss: 0.2386 - val_acc: 0.9016
Epoch 6/10
177/176 [==============================] - 31s - loss: 0.2276 - acc: 0.9058 - val_loss: 0.2354 - val_acc: 0.9029
Epoch 7/10
177/176 [==============================] - 31s - loss: 0.2249 - acc: 0.9066 - val_loss: 0.2250 - val_acc: 0.9075
Epoch 8/10
177/176 [==============================] - 31s - loss: 0.2231 - acc: 0.9083 - val_loss: 0.2446 - val_acc: 0.8969
Epoch 9/

In [39]:
y_pred = model1.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_2, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.763049753326
thresh: 0.1 	F2 score: 0.798936952927
thresh: 0.15 	F2 score: 0.8024707007
thresh: 0.2 	F2 score: 0.799249070495
thresh: 0.25 	F2 score: 0.787243646363
thresh: 0.3 	F2 score: 0.777298645551
thresh: 0.35 	F2 score: 0.766756612624


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [40]:
#continue with reduced learning rate
model1.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0005),
              metrics=['accuracy']) 
model1.fit_generator(datagen.flow(x_train,y_train_2, batch_size = 128), validation_data=(x_val, y_val_2),
                   epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
177/176 [==============================] - 34s - loss: 0.2131 - acc: 0.9126 - val_loss: 0.2130 - val_acc: 0.9145
Epoch 2/10
177/176 [==============================] - 31s - loss: 0.2110 - acc: 0.9139 - val_loss: 0.2218 - val_acc: 0.9074
Epoch 3/10
177/176 [==============================] - 31s - loss: 0.2100 - acc: 0.9138 - val_loss: 0.2084 - val_acc: 0.9153
Epoch 4/10
177/176 [==============================] - 31s - loss: 0.2086 - acc: 0.9144 - val_loss: 0.2073 - val_acc: 0.9175
Epoch 5/10
177/176 [==============================] - 31s - loss: 0.2063 - acc: 0.9151 - val_loss: 0.2082 - val_acc: 0.9154
Epoch 6/10
177/176 [==============================] - 31s - loss: 0.2068 - acc: 0.9152 - val_loss: 0.2089 - val_acc: 0.9142
Epoch 7/10
177/176 [==============================] - 31s - loss: 0.2056 - acc: 0.9159 - val_loss: 0.2156 - val_acc: 0.9128


In [41]:
y_pred = model1.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_2, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.770638243595
thresh: 0.1 	F2 score: 0.799896182667
thresh: 0.15 	F2 score: 0.80807109544
thresh: 0.2 	F2 score: 0.80575773335
thresh: 0.25 	F2 score: 0.79474363349
thresh: 0.3 	F2 score: 0.784058100791
thresh: 0.35 	F2 score: 0.773709917281


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [42]:
#continue with reduced learning rate
model1.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0002),
              metrics=['accuracy']) 
model1.fit_generator(datagen.flow(x_train,y_train_2, batch_size = 128), validation_data=(x_val, y_val_2),
                   epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
177/176 [==============================] - 35s - loss: 0.2009 - acc: 0.9176 - val_loss: 0.2040 - val_acc: 0.9179
Epoch 2/10
177/176 [==============================] - 31s - loss: 0.1985 - acc: 0.9186 - val_loss: 0.1996 - val_acc: 0.9198
Epoch 3/10
177/176 [==============================] - 31s - loss: 0.1980 - acc: 0.9183 - val_loss: 0.1982 - val_acc: 0.9192
Epoch 4/10
177/176 [==============================] - 31s - loss: 0.1972 - acc: 0.9191 - val_loss: 0.2010 - val_acc: 0.9193
Epoch 5/10
177/176 [==============================] - 31s - loss: 0.1975 - acc: 0.9190 - val_loss: 0.2010 - val_acc: 0.9182
Epoch 6/10
177/176 [==============================] - 31s - loss: 0.1968 - acc: 0.9196 - val_loss: 0.2008 - val_acc: 0.9190


In [43]:
y_pred = model1.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_2, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.779047528093
thresh: 0.1 	F2 score: 0.809476769439
thresh: 0.15 	F2 score: 0.820844177521
thresh: 0.2 	F2 score: 0.820803629605
thresh: 0.25 	F2 score: 0.813905763879
thresh: 0.3 	F2 score: 0.805927528302
thresh: 0.35 	F2 score: 0.792862623304


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [44]:
#continue with reduced learning rate
model1.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0001),
              metrics=['accuracy']) 
model1.fit_generator(datagen.flow(x_train,y_train_2, batch_size = 128), validation_data=(x_val, y_val_2),
                  epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
177/176 [==============================] - 35s - loss: 0.1944 - acc: 0.9205 - val_loss: 0.1979 - val_acc: 0.9202
Epoch 2/10
177/176 [==============================] - 31s - loss: 0.1933 - acc: 0.9207 - val_loss: 0.1993 - val_acc: 0.9185
Epoch 3/10
177/176 [==============================] - 31s - loss: 0.1931 - acc: 0.9209 - val_loss: 0.1969 - val_acc: 0.9194
Epoch 4/10
177/176 [==============================] - 31s - loss: 0.1920 - acc: 0.9211 - val_loss: 0.1987 - val_acc: 0.9187
Epoch 5/10
177/176 [==============================] - 31s - loss: 0.1931 - acc: 0.9206 - val_loss: 0.1990 - val_acc: 0.9189
Epoch 6/10
177/176 [==============================] - 31s - loss: 0.1929 - acc: 0.9207 - val_loss: 0.1978 - val_acc: 0.9208


In [45]:
y_pred = model1.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_2, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.786889746228
thresh: 0.1 	F2 score: 0.815301058606
thresh: 0.15 	F2 score: 0.823508906662
thresh: 0.2 	F2 score: 0.824182371993
thresh: 0.25 	F2 score: 0.818445853154
thresh: 0.3 	F2 score: 0.808495691192
thresh: 0.35 	F2 score: 0.79794724934


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [46]:
#continue with reduced learning rate
model1.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.00005),
              metrics=['accuracy']) 
model1.fit_generator(datagen.flow(x_train,y_train_2, batch_size = 128), validation_data=(x_val, y_val_2),
                   epochs=10, steps_per_epoch=x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
177/176 [==============================] - 35s - loss: 0.1911 - acc: 0.9214 - val_loss: 0.1978 - val_acc: 0.9196
Epoch 2/10
177/176 [==============================] - 31s - loss: 0.1911 - acc: 0.9220 - val_loss: 0.1960 - val_acc: 0.9198
Epoch 3/10
177/176 [==============================] - 31s - loss: 0.1910 - acc: 0.9219 - val_loss: 0.1979 - val_acc: 0.9187
Epoch 4/10
177/176 [==============================] - 31s - loss: 0.1909 - acc: 0.9212 - val_loss: 0.1967 - val_acc: 0.9197
Epoch 5/10
177/176 [==============================] - 31s - loss: 0.1908 - acc: 0.9219 - val_loss: 0.1965 - val_acc: 0.9209


In [47]:
y_pred = model1.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_2, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.790680590547
thresh: 0.1 	F2 score: 0.815580311007
thresh: 0.15 	F2 score: 0.823472020045
thresh: 0.2 	F2 score: 0.824594499052
thresh: 0.25 	F2 score: 0.819632282387
thresh: 0.3 	F2 score: 0.812134433882
thresh: 0.35 	F2 score: 0.794785413742


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [48]:
num_rare = np.sum(np.sum(y_train[:,:7],axis=1)>0)
print(num_rare)

2180


In [49]:
model1.save("32_net_2")
model1.save_weights("32_net_2_weights")

In [50]:
x_val = []
x_train = []
x_train = np.zeros((num_rare,32,32,3), np.float32)
y_train = []

df_train = pd.read_csv('train_v2.csv')

labels = ['blow_down',
 'bare_ground',
 'conventional_mine',
 'blooming',
 'artisinal_mine',
 'selective_logging',         
 'slash_burn', 
 'cultivation',
 'habitation',
 'road',
 'water',
 'haze',
 'partly_cloudy', 
 'cloudy',
 'agriculture',
 'clear',
 'primary',]

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

i=0

for f, tags in tqdm(df_train.values[:40479], miniters=1000):    
    img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    if(np.sum(targets[:7])>0):
        x_train[i,:,:,:] = np.array(cv2.resize(img, (32, 32)),np.float32)/255.#139 minimum size for inception
        i+=1
        y_train.append(targets)
    
y_train = np.array(y_train, np.uint8)

print(x_train.shape)
print(y_train.shape)

100%|██████████| 40479/40479 [00:52<00:00, 767.65it/s]

(2180, 32, 32, 3)
(2180, 17)


In [51]:
x_train -= train_mean
x_train, x_val, y_train_3, y_val_3 = train_test_split(x_train,y_train[:,:7],test_size=0.1)
print(x_train.shape)
print(y_train_3.shape)
print(x_val.shape)
print(y_val_3.shape)

(1962, 32, 32, 3)
(1962, 7)
(218, 32, 32, 3)
(218, 7)


In [54]:
#model for the rarer classes
from keras.models import load_model
model2 = load_model("32_net_2")
model2.pop()
model2.add(Dense(7, activation='sigmoid'))

for layer in model2.layers[:-1]:
    layer.trainable = False
model2.layers[-1].trainable = True

In [55]:
model2.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])

model2.fit_generator(datagen.flow(x_train,y_train_3, batch_size = 128), validation_data=(x_val, y_val_3),
                  epochs=10, steps_per_epoch=10*x_train.shape[0]/ 128, callbacks=callbacks,
                  )#more steps per epoch to compensate for fewer images

Epoch 1/10
154/153 [==============================] - 12s - loss: 0.3063 - acc: 0.8818 - val_loss: 0.2302 - val_acc: 0.9135
Epoch 2/10
154/153 [==============================] - 8s - loss: 0.2429 - acc: 0.9010 - val_loss: 0.2143 - val_acc: 0.9161
Epoch 3/10
154/153 [==============================] - 8s - loss: 0.2356 - acc: 0.9027 - val_loss: 0.2131 - val_acc: 0.9181
Epoch 4/10
154/153 [==============================] - 8s - loss: 0.2319 - acc: 0.9031 - val_loss: 0.2069 - val_acc: 0.9168
Epoch 5/10
154/153 [==============================] - 8s - loss: 0.2263 - acc: 0.9056 - val_loss: 0.2112 - val_acc: 0.9155
Epoch 6/10
154/153 [==============================] - 8s - loss: 0.2263 - acc: 0.9052 - val_loss: 0.2087 - val_acc: 0.9122
Epoch 7/10
154/153 [==============================] - 8s - loss: 0.2242 - acc: 0.9060 - val_loss: 0.2088 - val_acc: 0.9128


In [56]:
y_pred = model2.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_3, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.723310265168
thresh: 0.1 	F2 score: 0.753589300608
thresh: 0.15 	F2 score: 0.773145114315
thresh: 0.2 	F2 score: 0.761868355905
thresh: 0.25 	F2 score: 0.751347021989
thresh: 0.3 	F2 score: 0.74872579001
thresh: 0.35 	F2 score: 0.719673802243


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [57]:
model2.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0005),
              metrics=['accuracy']) 
model2.fit_generator(datagen.flow(x_train,y_train_3, batch_size = 128), validation_data=(x_val, y_val_3),
                  epochs=10, steps_per_epoch=10*x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
154/153 [==============================] - 12s - loss: 0.2200 - acc: 0.9080 - val_loss: 0.1999 - val_acc: 0.9181
Epoch 2/10
154/153 [==============================] - 8s - loss: 0.2186 - acc: 0.9077 - val_loss: 0.2013 - val_acc: 0.9187
Epoch 3/10
154/153 [==============================] - 8s - loss: 0.2182 - acc: 0.9078 - val_loss: 0.2022 - val_acc: 0.9194
Epoch 4/10
154/153 [==============================] - 8s - loss: 0.2168 - acc: 0.9084 - val_loss: 0.2010 - val_acc: 0.9214


In [58]:
y_pred = model2.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_3, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.735531924752
thresh: 0.1 	F2 score: 0.768075578855
thresh: 0.15 	F2 score: 0.770314547837
thresh: 0.2 	F2 score: 0.76878549585
thresh: 0.25 	F2 score: 0.763106159895
thresh: 0.3 	F2 score: 0.740752876074
thresh: 0.35 	F2 score: 0.729831076161


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [59]:
model2.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0002),
              metrics=['accuracy']) 
model2.fit_generator(datagen.flow(x_train,y_train_3, batch_size = 128), validation_data=(x_val, y_val_3),
                  epochs=10, steps_per_epoch=10*x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
154/153 [==============================] - 12s - loss: 0.2177 - acc: 0.9078 - val_loss: 0.2014 - val_acc: 0.9214
Epoch 2/10
154/153 [==============================] - 8s - loss: 0.2153 - acc: 0.9091 - val_loss: 0.2008 - val_acc: 0.9220
Epoch 3/10
154/153 [==============================] - 8s - loss: 0.2165 - acc: 0.9083 - val_loss: 0.2012 - val_acc: 0.9187
Epoch 4/10
154/153 [==============================] - 8s - loss: 0.2149 - acc: 0.9089 - val_loss: 0.2006 - val_acc: 0.9214
Epoch 5/10
154/153 [==============================] - 8s - loss: 0.2141 - acc: 0.9090 - val_loss: 0.1997 - val_acc: 0.9201
Epoch 6/10
154/153 [==============================] - 8s - loss: 0.2131 - acc: 0.9097 - val_loss: 0.2006 - val_acc: 0.9194
Epoch 7/10
154/153 [==============================] - 8s - loss: 0.2144 - acc: 0.9089 - val_loss: 0.1996 - val_acc: 0.9207
Epoch 8/10
154/153 [==============================] - 8s - loss: 0.2135 - acc: 0.9093 - val_loss: 0.1999 - val_acc: 0.9187
Epoch 9/10
154/

In [60]:
y_pred = model2.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_3, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.739354554722
thresh: 0.1 	F2 score: 0.75962276765
thresh: 0.15 	F2 score: 0.777752128096
thresh: 0.2 	F2 score: 0.778360273773
thresh: 0.25 	F2 score: 0.769659239843
thresh: 0.3 	F2 score: 0.758082131935
thresh: 0.35 	F2 score: 0.737257900102


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [61]:
model2.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.0001),
              metrics=['accuracy']) 
model2.fit_generator(datagen.flow(x_train,y_train_3, batch_size = 128), validation_data=(x_val, y_val_3),
                  epochs=10, steps_per_epoch=10*x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
154/153 [==============================] - 12s - loss: 0.2134 - acc: 0.9090 - val_loss: 0.1986 - val_acc: 0.9214
Epoch 2/10
154/153 [==============================] - 8s - loss: 0.2105 - acc: 0.9110 - val_loss: 0.1998 - val_acc: 0.9207
Epoch 3/10
154/153 [==============================] - 8s - loss: 0.2129 - acc: 0.9098 - val_loss: 0.1990 - val_acc: 0.9207
Epoch 4/10
154/153 [==============================] - 8s - loss: 0.2112 - acc: 0.9100 - val_loss: 0.1987 - val_acc: 0.9207


In [62]:
y_pred = model2.predict(x_val,batch_size=128)
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    print("thresh:",thresh,"\tF2 score:",fbeta_score(y_val_3, np.array(y_pred)>thresh, beta=2, average='samples'))

thresh: 0.05 	F2 score: 0.738817566226
thresh: 0.1 	F2 score: 0.763055687941
thresh: 0.15 	F2 score: 0.778933668269
thresh: 0.2 	F2 score: 0.770715013834
thresh: 0.25 	F2 score: 0.77126110383
thresh: 0.3 	F2 score: 0.757317605941
thresh: 0.35 	F2 score: 0.734199796126


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [63]:
model2.compile(loss='binary_crossentropy', 
              optimizer=Adam(lr=0.00005),
              metrics=['accuracy']) 
model2.fit_generator(datagen.flow(x_train,y_train_3, batch_size = 128), validation_data=(x_val, y_val_3),
                  epochs=10, steps_per_epoch=10*x_train.shape[0]/ 128, callbacks=callbacks,
                  )

Epoch 1/10
154/153 [==============================] - 12s - loss: 0.2124 - acc: 0.9103 - val_loss: 0.1995 - val_acc: 0.9201
Epoch 2/10
154/153 [==============================] - 8s - loss: 0.2133 - acc: 0.9096 - val_loss: 0.1986 - val_acc: 0.9207
Epoch 3/10
154/153 [==============================] - 8s - loss: 0.2117 - acc: 0.9107 - val_loss: 0.1987 - val_acc: 0.9214
Epoch 4/10
154/153 [==============================] - 8s - loss: 0.2130 - acc: 0.9086 - val_loss: 0.1991 - val_acc: 0.9194
Epoch 5/10
154/153 [==============================] - 8s - loss: 0.2128 - acc: 0.9095 - val_loss: 0.1991 - val_acc: 0.9220


In [64]:
y_pred = model2.predict(x_val,batch_size=128)
bestthresh = 0
bestF2score = 0
for thresh in [0.05,0.1,0.15,0.2,0.25,0.3,0.35]:
    F2score = fbeta_score(y_val_3, np.array(y_pred)>thresh, beta=2, average='samples')
    print("thresh:",thresh,"\tF2 score:",F2score)
    if F2score > bestF2score:
        bestthresh = thresh
        bestF2score = F2score    

thresh: 0.05 	F2 score: 0.740028065716
thresh: 0.1 	F2 score: 0.763820213935
thresh: 0.15 	F2 score: 0.777404616281
thresh: 0.2 	F2 score: 0.774537643804
thresh: 0.25 	F2 score: 0.765618173875
thresh: 0.3 	F2 score: 0.753494975972
thresh: 0.35 	F2 score: 0.738022426096


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [65]:
total,tp,tn,fp,fn = multilabelmetrics(y_val_3,np.array(y_pred)>bestthresh)
d = {'Total':total,'TP':tp,'TN':tn,'FP':fp,'FN':fn}
pd.DataFrame(d, index=labels[:7])

,FN,FP,TN,TP,Total
blow_down,4,11,198,5,9
bare_ground,4,71,65,78,82
conventional_mine,2,12,201,3,5
blooming,5,16,155,42,47
artisinal_mine,1,23,166,28,29
selective_logging,7,40,149,22,29
slash_burn,3,23,172,20,23


In [66]:
model2.save("32_net_3")
model2.save_weights("32_net_3_weights")

In [82]:
#F2 score on training set
x_val = []
x_train = []
x_train = np.zeros((40479,32,32,3), np.float32)
y_train = []

df_train = pd.read_csv('train_v2.csv')

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

i=0

for f, tags in tqdm(df_train.values[:40479], miniters=1000):    
    img = cv2.imread('train-jpg/{}.jpg'.format(f))
    targets = np.zeros(17)
    for t in tags.split(' '):
        targets[label_map[t]] = 1 
    x_train[i,:,:,:] = np.array(cv2.resize(img, (32, 32)),np.float32)/255.#139 minimum size for inception
    i+=1
    y_train.append(targets)

y_train = np.array(y_train, np.uint8)

print(x_train.shape)
print(y_train.shape)


100%|██████████| 40479/40479 [00:54<00:00, 744.98it/s]

(40479, 32, 32, 3)
(40479, 17)


In [83]:
x_train -= train_mean

In [84]:
y = model.predict(x_train,batch_size=128)
y1 = model1.predict(x_train,batch_size=128)
y2 = model2.predict(x_train,batch_size=128)

In [85]:
print(y.shape)
print(y1.shape)
print(y2.shape)

(40479, 4)
(40479, 8)
(40479, 7)


In [86]:
def combine_predictions_2(y,y1,y2,thresh):
    y_pred = np.zeros((y.shape[0],17),np.uint8)
    y_bool = np.array((y[:,1:] > thresh[0:3]),np.uint8)
    y1_bool = np.array((y1[:,1:] > thresh[3:10])*np.tile(y[:,0]>thresh[17],(7,1)).T,np.uint8)
    y2_bool = np.array((y2 > thresh[10:17])*np.tile(y1[:,0]>thresh[18],(7,1)).T,np.uint8)
    y_pred[:,:7] = y2_bool
    y_pred[:,7:14] = y1_bool
    y_pred[:,14:17] = y_bool
    y_pred[((np.nonzero(y1[:,-1]>thresh[19]))[0]),:] = 0
    y_pred[((np.nonzero(y1[:,-1]>thresh[19]))[0]),13] = 1
    return y_pred

def optimise_f2_thresholds(x_init,y, y1,y2, ytrue, num_thresh=20, verbose=True, resolution=100):
  def mf(x):
    p2 = combine_predictions_2(y, y1,y2,x)
    score = fbeta_score(ytrue, p2, beta=2, average='samples')
    return score

  x = list(x_init)
  for i in range(num_thresh):
    best_i2 = 0
    best_score = 0
    for i2 in range(resolution):
      i2 /= resolution
      x[i] = i2
      score = mf(x)
      if score > best_score:
        best_i2 = i2
        best_score = score
    x[i] = best_i2
    if verbose:
      print(i, best_i2, best_score)

  return x

In [87]:
thresh = optimise_f2_thresholds([0.2]*20,y,y1,y2,y_train)

0 0.22 0.896853143732
1 0.17 0.896898377013
2 0.26 0.896918140142
3 0.23 0.896974259947
4 0.23 0.897018173566
5 0.25 0.89715743809
6 0.32 0.897456768996
7 0.21 0.89747181798
8 0.26 0.897521828877
9 0.09 0.897772112092
10 0.58 0.89838891842
11 0.55 0.898678831248
12 0.33 0.89871037124
13 0.98 0.911761929332
14 0.34 0.911815461304
15 0.55 0.913958463848
16 0.64 0.914217217736
17 0.3 0.914336717765
18 0.26 0.914467271068
19 0.79 0.923470509553


In [88]:
thresh = optimise_f2_thresholds(thresh,y,y1,y2,y_train)

0 0.22 0.923470509553
1 0.17 0.923470509553
2 0.21 0.923500804629
3 0.23 0.923500804629
4 0.23 0.923500804629
5 0.25 0.923500804629
6 0.26 0.923579304446
7 0.19 0.923602453014
8 0.26 0.923602453014
9 0.09 0.923602453014
10 0.52 0.923604603154
11 0.32 0.923621863812
12 0.28 0.92362471222
13 0.98 0.92362471222
14 0.34 0.92362471222
15 0.55 0.92362471222
16 0.5 0.923624933508
17 0.3 0.923624933508
18 0.27 0.923633580271
19 0.79 0.923633580271


/home/cs231n/myVE35/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


In [89]:
y_pred = combine_predictions_2(y,y1,y2,thresh)
total,tp,tn,fp,fn = multilabelmetrics(y_train,y_pred)
d = {'Total':total,'TP':tp,'TN':tn,'FP':fp,'FN':fn}
pd.DataFrame(d, index=labels)

,FN,FP,TN,TP,Total
blow_down,90,16,40365,8,98
bare_ground,437,876,38741,425,862
conventional_mine,58,45,40334,42,100
blooming,332,0,40147,0,332
artisinal_mine,72,190,39950,267,339
selective_logging,236,193,39946,104,340
slash_burn,198,21,40249,11,209
cultivation,1087,3821,32181,3390,4477
habitation,874,2390,34429,2786,3660
road,851,3894,28514,7220,8071


In [90]:
#Test set
x_train = []
x_val = []
x_test = np.zeros((61191,32,32,3), np.float32)
y_train = []

df_test = pd.read_csv('sample_submission_v2.csv')

i = 0 
for f, tags in tqdm(df_test.values, miniters=1000):
    img = cv2.imread('test-jpg/{}.jpg'.format(f))
    x_test[i,:,:,:] = np.array(cv2.resize(img, (32, 32)),np.float32)/255.#139 minimum size for inception
    i+=1
print(x_test.shape)

100%|██████████| 61191/61191 [02:03<00:00, 494.20it/s]

(61191, 32, 32, 3)


In [91]:
x_test -= train_mean

y = model.predict(x_test,batch_size=128)
y1 = model1.predict(x_test,batch_size=128)
y2 = model2.predict(x_test,batch_size=128)

In [92]:
y_pred = combine_predictions_2(y,y1,y2,thresh)
labels_np = np.array(labels)
preds = [' '.join(labels_np[np.array(y_pred[i,:],bool)]) for i in range(y_pred.shape[0])]
subm = pd.DataFrame()
subm['image_name'] = df_test.values[:,0]
subm['tags'] = preds
subm.to_csv('submission_32_net_1.csv', index=False)
#test set score:0.91567